<a href="https://colab.research.google.com/github/danimacaya/AIInternational/blob/main/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


current_dir = "/content/drive/MyDrive/Assignment_25_26"
os.chdir(current_dir)

meta_path = os.path.join(current_dir, "training_metadata.xlsx")
df = pd.read_excel(meta_path)  # columns: US, MASK, LABEL
print(df.head())
print(df['LABEL'].value_counts())  # 0=benign, 1=malignant, 2=normal0}2


Mounted at /content/drive
      US        MASK  LABEL
0  1.png  1_mask.png      2
1  2.png  2_mask.png      2
2  3.png  3_mask.png      2
3  4.png  4_mask.png      2
4  5.png  5_mask.png      2
LABEL
0    679
2    460
1    364
Name: count, dtype: int64


In [ ]:
import re

def extract_patient_id(us_name: str) -> str:
    base = os.path.splitext(us_name)[0]  # e.g. '1040_3'
    # Split on first underscore, take left part
    return base.split('_')[0]

df['PATIENT_ID'] = df['US'].astype(str).apply(extract_patient_id)

print(df[['US', 'PATIENT_ID']].head(20))
print(df['PATIENT_ID'].nunique(), "unique patients")


        US PATIENT_ID
0    1.png          1
1    2.png          2
2    3.png          3
3    4.png          4
4    5.png          5
5    6.png          6
6    7.png          7
7    8.png          8
8    9.png          9
9   10.png         10
10  11.png         11
11  12.png         12
12  13.png         13
13  14.png         14
14  15.png         15
15  16.png         16
16  17.png         17
17  18.png         18
18  19.png         19
19  20.png         20
968 unique patients


In [ ]:
label_map = {0: 'benign', 1: 'malignant', 2: 'normal'}
df['LABEL_NAME'] = df['LABEL'].map(label_map)

print("Global label counts:")
print(df['LABEL_NAME'].value_counts())


Global label counts:
LABEL_NAME
benign       679
normal       460
malignant    364
Name: count, dtype: int64


In [ ]:
per_patient = df.groupby('PATIENT_ID')['LABEL'].agg(['count', 'nunique'])
print(per_patient.head(20))


            count  nunique
PATIENT_ID                
1               1        1
10              1        1
100             1        1
1000            1        1
1001            1        1
1003            1        1
1007            1        1
1008            1        1
1009            1        1
101             1        1
1010            1        1
1012            1        1
1015            1        1
1016            1        1
102             1        1
1020            1        1
1021            1        1
1025            1        1
1026            1        1
103             1        1


In [ ]:
# Representative label per patient (mode)
rep = df.groupby('PATIENT_ID')['LABEL'].agg(lambda x: x.value_counts().idxmax())
rep = rep.reset_index().rename(columns={'LABEL': 'REP_LABEL'})
print(rep['REP_LABEL'].value_counts())


REP_LABEL
0    537
1    286
2    145
Name: count, dtype: int64


In [ ]:
from sklearn.model_selection import StratifiedGroupKFold

X = rep['PATIENT_ID'].values
y = rep['REP_LABEL'].values
groups = rep['PATIENT_ID'].values

cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
train_pids, val_pids = next(cv.split(X, y, groups))

train_patient_ids = rep.iloc[train_pids]['PATIENT_ID']
val_patient_ids   = rep.iloc[val_pids]['PATIENT_ID']

df_train = df[df['PATIENT_ID'].isin(train_patient_ids)].reset_index(drop=True)
df_val   = df[df['PATIENT_ID'].isin(val_patient_ids)].reset_index(drop=True)

print("Train images:", len(df_train), "Val images:", len(df_val))
print("Train label distribution:\n", df_train['LABEL_NAME'].value_counts())
print("Val label distribution:\n", df_val['LABEL_NAME'].value_counts())


Train images: 1177 Val images: 326
Train label distribution:
 LABEL_NAME
benign       542
normal       355
malignant    280
Name: count, dtype: int64
Val label distribution:
 LABEL_NAME
benign       137
normal       105
malignant     84
Name: count, dtype: int64


In [ ]:
df.to_csv("metadata_full_with_patient.csv", index=False)
df_train.to_csv("metadata_train.csv", index=False)
df_val.to_csv("metadata_val.csv", index=False)



In [ ]:
!pip install -q timm albumentations==1.4.3

import os
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.metrics import f1_score, classification_report, confusion_matrix
from tqdm.auto import tqdm

# ----------------- CONFIG -----------------
class Config:
    img_dir = 'training_images'           # folder with .png images [file:1]
    train_meta = 'metadata_train.csv'     # from preprocessing step
    val_meta   = 'metadata_val.csv'

    output_dir = 'outputs_cls'
    model_name = 'tf_efficientnetv2_s'
    num_classes = 3                       # 0=benign,1=malignant,2=normal [file:1]
    img_size = 384

    epochs = 40
    batch_size = 16
    lr = 1e-4
    weight_decay = 1e-5
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    num_workers = 4

    seed = 42

config = Config()
os.makedirs(config.output_dir, exist_ok=True)

# ----------------- SEED -----------------
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(config.seed)

# ----------------- TRANSFORMS -----------------
def get_transforms(phase='train'):
    if phase == 'train':
        return A.Compose([
            A.Resize(config.img_size, config.img_size),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.2),
            A.RandomRotate90(p=0.3),
            A.ShiftScaleRotate(shift_limit=0.05,
                               scale_limit=0.1,
                               rotate_limit=15,
                               border_mode=0,
                               p=0.4),
            A.OneOf([
                A.GaussNoise(var_limit=(5.0, 25.0), p=0.5),
                A.GaussianBlur(blur_limit=(3, 5), p=0.5),
            ], p=0.3),
            A.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])
    else:
        return A.Compose([
            A.Resize(config.img_size, config.img_size),
            A.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])

# ----------------- DATASET -----------------
class BreastUSDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_name = row['US']          # e.g. '1.png'
        label = int(row['LABEL'])     # 0/1/2

        img_path = os.path.join(self.img_dir, img_name)
        # images are grayscale; convert to RGB by repetition for timm models
        image = Image.open(img_path).convert('L')
        image = np.array(image)
        image = np.stack([image, image, image], axis=2)  # H,W -> H,W,3

        if self.transform:
            aug = self.transform(image=image)
            image = aug['image']

        return image, label

# ----------------- MODEL -----------------
class USClassifier(nn.Module):
    def __init__(self, model_name=config.model_name,
                 num_classes=config.num_classes, pretrained=True):
        super().__init__()
        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=0
        )
        in_features = self.backbone.num_features
        self.classifier = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(in_features, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        feats = self.backbone(x)
        out = self.classifier(feats)
        return out

# ----------------- TRAIN / VALIDATE -----------------
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    all_preds, all_labels = [], []

    pbar = tqdm(loader, desc='Train', leave=False)
    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = outputs.argmax(1)
        all_preds.extend(preds.detach().cpu().numpy())
        all_labels.extend(labels.detach().cpu().numpy())

        pbar.set_postfix(loss=running_loss / len(pbar))

    epoch_loss = running_loss / len(loader)
    epoch_f1 = f1_score(all_labels, all_preds, average='macro')
    return epoch_loss, epoch_f1

def validate_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds, all_labels = [], []

    with torch.no_grad():
        pbar = tqdm(loader, desc='Val', leave=False)
        for images, labels in pbar:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            preds = outputs.argmax(1)
            all_preds.extend(preds.detach().cpu().numpy())
            all_labels.extend(labels.detach().cpu().numpy())

    epoch_loss = running_loss / len(loader)
    epoch_f1 = f1_score(all_labels, all_preds, average='macro')
    return epoch_loss, epoch_f1, np.array(all_preds), np.array(all_labels)

# ----------------- MAIN TRAINING -----------------
def main():
    print(f"Device: {config.device}")
    print(f"Backbone: {config.model_name}")

    train_df = pd.read_csv(config.train_meta)
    val_df   = pd.read_csv(config.val_meta)

    print("Train size:", len(train_df), "Val size:", len(val_df))
    print("Train label counts:\n", train_df['LABEL'].value_counts())
    print("Val label counts:\n", val_df['LABEL'].value_counts())

    train_tfms = get_transforms('train')
    val_tfms   = get_transforms('val')

    train_ds = BreastUSDataset(train_df, config.img_dir, transform=train_tfms)
    val_ds   = BreastUSDataset(val_df,   config.img_dir, transform=val_tfms)

    config.num_workers = 0  # or even 0
    train_loader = DataLoader(
        train_ds,
        batch_size=config.batch_size,
        shuffle=True,
        num_workers=config.num_workers,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_ds,
        batch_size=config.batch_size,
        shuffle=False,
        num_workers=config.num_workers,
        pin_memory=True
    )

    model = USClassifier().to(config.device)

    # class weights to mitigate imbalance
    class_counts = train_df['LABEL'].value_counts().sort_index().values
    class_weights = 1.0 / (class_counts + 1e-6)
    class_weights = class_weights / class_weights.sum() * len(class_counts)
    class_weights = torch.tensor(class_weights, dtype=torch.float32).to(config.device)

    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.AdamW(model.parameters(),
                            lr=config.lr,
                            weight_decay=config.weight_decay)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=config.epochs, eta_min=1e-6
    )

    best_f1 = 0.0
    best_path = os.path.join(config.output_dir, 'best_cls_model.pth')
    history = {'train_loss': [], 'val_loss': [], 'train_f1': [], 'val_f1': []}

    for epoch in range(config.epochs):
        print(f"\nEpoch {epoch+1}/{config.epochs}")
        train_loss, train_f1 = train_epoch(model, train_loader, criterion, optimizer, config.device)
        val_loss, val_f1, val_preds, val_labels = validate_epoch(model, val_loader, criterion, config.device)

        scheduler.step()

        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_f1'].append(train_f1)
        history['val_f1'].append(val_f1)

        print(f"Train Loss: {train_loss:.4f} | F1: {train_f1:.4f}")
        print(f"Val   Loss: {val_loss:.4f} | F1: {val_f1:.4f}")

        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'history': history,
                'best_f1': best_f1,
            }, best_path)
            print(f"✓ Saved best model (F1={best_f1:.4f})")

    # reload best model for final report metrics
    ckpt = torch.load(best_path, map_location=config.device)
    model.load_state_dict(ckpt['model_state_dict'])
    history = ckpt['history']
    print("\nBest Val F1:", ckpt['best_f1'])

    _, _, val_preds, val_labels = validate_epoch(model, val_loader, criterion, config.device)
    print("\nClassification report (val):")
    target_names = ['benign (0)', 'malignant (1)', 'normal (2)']
    print(classification_report(val_labels, val_preds, target_names=target_names))

    cm = confusion_matrix(val_labels, val_preds, labels=[0,1,2])
    print("Confusion matrix (val):\n", cm)

    # save history for plotting curves for the report
    hist_df = pd.DataFrame(history)
    hist_df.to_csv(os.path.join(config.output_dir, 'cls_training_history.csv'), index=False)

if __name__ == "__main__":
    main()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 8.9 MB/s eta 0:00:00
Device: cuda
Backbone: tf_efficientnetv2_s
Train size: 1177 Val size: 326
Train label counts:
 LABEL
0    542
2    355
1    280
Name: count, dtype: int64
Val label counts:
 LABEL
0    137
2    105
1     84
Name: count, dtype: int64


model.safetensors:   0%|          | 0.00/86.5M [00:00<?, ?B/s]


Epoch 1/40


Train:   0%|          | 0/74 [00:00<?, ?it/s]

Val:   0%|          | 0/21 [00:00<?, ?it/s]

...

Train:   0%|          | 0/74 [00:00<?, ?it/s]

Val:   0%|          | 0/21 [00:00<?, ?it/s]

Train Loss: 0.0281 | F1: 0.9871
Val   Loss: 0.8971 | F1: 0.7963

Best Val F1: 0.8201512857358703


Val:   0%|          | 0/21 [00:00<?, ?it/s]


Classification report (val):
               precision    recall  f1-score   support

   benign (0)       0.82      0.86      0.84       137
malignant (1)       0.75      0.71      0.73        84
   normal (2)       0.90      0.88      0.89       105

     accuracy                           0.83       326
    macro avg       0.82      0.82      0.82       326
 weighted avg       0.83      0.83      0.83       326

Confusion matrix (val):
 [[118  12   7]
 [ 21  60   3]
 [  5   8  92]]


In [ ]:
!pip install -q albumentations==1.4.3

import os
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import balanced_accuracy_score

from sklearn.metrics import f1_score
from tqdm.auto import tqdm

# ----------------- CONFIG -----------------
class SegConfig:
    img_dir   = 'training_images'        # images + *_mask.png
    train_meta = 'metadata_train.csv'    # same as used for classifier
    val_meta   = 'metadata_val.csv'

    output_dir = 'outputs_seg_bm'
    img_size   = 384

    epochs     = 60
    batch_size = 8
    lr         = 1e-4
    weight_decay = 1e-5
    device     = 'cuda' if torch.cuda.is_available() else 'cpu'
    num_workers = 0   # safest in Colab

    seed = 42

seg_cfg = SegConfig()
os.makedirs(seg_cfg.output_dir, exist_ok=True)

# ----------------- SEED -----------------
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(seg_cfg.seed)

# ----------------- TRANSFORMS -----------------
def get_seg_transforms(phase='train'):
    if phase == 'train':
        return A.Compose([
            A.Resize(seg_cfg.img_size, seg_cfg.img_size),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.2),
            A.RandomRotate90(p=0.3),
            A.ShiftScaleRotate(shift_limit=0.05,
                               scale_limit=0.1,
                               rotate_limit=15,
                               border_mode=0,
                               p=0.4),
            A.OneOf([
                A.GaussNoise(var_limit=(5.0, 25.0), p=0.5),
                A.GaussianBlur(blur_limit=(3, 5), p=0.5),
            ], p=0.3),
            A.Normalize(mean=[0.0], std=[1.0]),
            ToTensorV2(),
        ])
    else:
        return A.Compose([
            A.Resize(seg_cfg.img_size, seg_cfg.img_size),
            A.Normalize(mean=[0.0], std=[1.0]),
            ToTensorV2(),
        ])

# ----------------- DATASET -----------------
class BMSegmentationDataset(Dataset):
    """
    2-class lesion segmentation (B+M vs background).
    Uses LABEL ∈ {0,1} and MASK as foreground.
    """

    def __init__(self, df, img_dir, transform=None):
        # keep only benign + malignant
        df = df[df['LABEL'].isin([0,1])].reset_index(drop=True)
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_name  = row['US']
        mask_name = row['MASK']

        img_path  = os.path.join(self.img_dir, img_name)
        mask_path = os.path.join(self.img_dir, mask_name)

        # grayscale image
        image = Image.open(img_path).convert('L')
        image = np.array(image)  # H,W

        # mask: binary 0/1 foreground
        mask = Image.open(mask_path).convert('L')
        mask = np.array(mask)
        mask = (mask > 0).astype('float32')  # H,W

        # Albumentations expects dictionary
        if self.transform:
            aug = self.transform(image=image, mask=mask)
            image = aug['image']        # 1xHxW
            mask  = aug['mask']         # HxW
        else:
            image = torch.from_numpy(image).unsqueeze(0).float()
            mask  = torch.from_numpy(mask).float()

        # make mask shape 1xHxW
        if mask.ndim == 2:
            mask = mask.unsqueeze(0)

        return image, mask

# ----------------- U-NET MODEL -----------------
class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.net(x)

class UNet(nn.Module):
    def __init__(self, n_channels=1, n_classes=1):
        super().__init__()
        self.inc   = DoubleConv(n_channels, 64)
        self.down1 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(64, 128))
        self.down2 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(128, 256))
        self.down3 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(256, 512))
        self.down4 = nn.Sequential(nn.MaxPool2d(2), DoubleConv(512, 512))

        self.up1   = nn.ConvTranspose2d(512, 512, kernel_size=2, stride=2)
        self.conv1 = DoubleConv(512 + 512, 256)
        self.up2   = nn.ConvTranspose2d(256, 256, kernel_size=2, stride=2)
        self.conv2 = DoubleConv(256 + 256, 128)
        self.up3   = nn.ConvTranspose2d(128, 128, kernel_size=2, stride=2)
        self.conv3 = DoubleConv(128 + 128, 64)
        self.up4   = nn.ConvTranspose2d(64, 64, kernel_size=2, stride=2)
        self.conv4 = DoubleConv(64 + 64, 64)

        self.outc  = nn.Conv2d(64, n_classes, kernel_size=1)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)

        x = self.up1(x5)
        x = torch.cat([x, x4], dim=1)
        x = self.conv1(x)
        x = self.up2(x)
        x = torch.cat([x, x3], dim=1)
        x = self.conv2(x)
        x = self.up3(x)
        x = torch.cat([x, x2], dim=1)
        x = self.conv3(x)
        x = self.up4(x)
        x = torch.cat([x, x1], dim=1)
        x = self.conv4(x)
        logits = self.outc(x)  # Bx1xHxW
        return logits

# ----------------- LOSSES & METRICS -----------------
class DiceBCELoss(nn.Module):
    """Dice + BCE for binary segmentation."""
    def __init__(self, smooth=1.0):
        super().__init__()
        self.bce = nn.BCEWithLogitsLoss()
        self.smooth = smooth

    def forward(self, logits, targets):
        bce = self.bce(logits, targets)

        probs = torch.sigmoid(logits)
        probs_flat = probs.view(-1)
        targets_flat = targets.view(-1)

        intersection = (probs_flat * targets_flat).sum()
        dice = (2. * intersection + self.smooth) / (
            probs_flat.sum() + targets_flat.sum() + self.smooth
        )
        dice_loss = 1 - dice
        return bce + dice_loss

def dice_coeff(preds, targets, threshold=0.5, eps=1e-7):
    probs = torch.sigmoid(preds)
    preds_bin = (probs > threshold).float()

    intersection = (preds_bin * targets).sum(dim=(1,2,3))
    union = preds_bin.sum(dim=(1,2,3)) + targets.sum(dim=(1,2,3))
    dice = (2 * intersection + eps) / (union + eps)
    return dice.mean().item()

# ----------------- TRAIN / VALID -----------------
def train_seg_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    running_dice = 0.0
    n_batches = 0

    pbar = tqdm(loader, desc='Seg Train', leave=False)
    for images, masks in pbar:
        images = images.to(device)
        masks  = masks.to(device)

        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, masks)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_dice += dice_coeff(logits.detach(), masks.detach())
        n_batches += 1

        pbar.set_postfix(loss=running_loss/n_batches,
                         dice=running_dice/n_batches)

    return running_loss/n_batches, running_dice/n_batches

def valid_seg_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_dice = 0.0
    n_batches = 0

    with torch.no_grad():
        pbar = tqdm(loader, desc='Seg Val', leave=False)
        for images, masks in pbar:
            images = images.to(device)
            masks  = masks.to(device)

            logits = model(images)
            loss = criterion(logits, masks)

            running_loss += loss.item()
            running_dice += dice_coeff(logits, masks)
            n_batches += 1

    return running_loss/n_batches, running_dice/n_batches

# ----------------- MAIN -----------------
def main():
    print("Device:", seg_cfg.device)

    train_df = pd.read_csv(seg_cfg.train_meta)
    val_df   = pd.read_csv(seg_cfg.val_meta)

    print("Original train size:", len(train_df), "val size:", len(val_df))
    print("Train label counts:\n", train_df['LABEL'].value_counts())
    print("Val label counts:\n",   val_df['LABEL'].value_counts())

    # Datasets only on B/M (0,1)
    train_ds = BMSegmentationDataset(train_df, seg_cfg.img_dir,
                                     transform=get_seg_transforms('train'))
    val_ds   = BMSegmentationDataset(val_df, seg_cfg.img_dir,
                                     transform=get_seg_transforms('val'))

    print("Seg train size (B+M):", len(train_ds), "Seg val size (B+M):", len(val_ds))

    train_loader = DataLoader(
        train_ds, batch_size=seg_cfg.batch_size, shuffle=True,
        num_workers=seg_cfg.num_workers, pin_memory=False
    )
    val_loader = DataLoader(
        val_ds, batch_size=seg_cfg.batch_size, shuffle=False,
        num_workers=seg_cfg.num_workers, pin_memory=False
    )

    model = UNet(n_channels=1, n_classes=1).to(seg_cfg.device)
    criterion = DiceBCELoss()
    optimizer = optim.AdamW(model.parameters(),
                            lr=seg_cfg.lr,
                            weight_decay=seg_cfg.weight_decay)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=seg_cfg.epochs, eta_min=1e-6
    )

    best_dice = 0.0
    best_path = os.path.join(seg_cfg.output_dir, 'best_seg_bm.pth')
    history = {'train_loss': [], 'val_loss': [],
               'train_dice': [], 'val_dice': []}

    for epoch in range(seg_cfg.epochs):
        print(f"\nEpoch {epoch+1}/{seg_cfg.epochs}")
        tr_loss, tr_dice = train_seg_epoch(model, train_loader, criterion, optimizer, seg_cfg.device)
        val_loss, val_dice = valid_seg_epoch(model, val_loader, criterion, seg_cfg.device)

        scheduler.step()

        history['train_loss'].append(tr_loss)
        history['val_loss'].append(val_loss)
        history['train_dice'].append(tr_dice)
        history['val_dice'].append(val_dice)

        print(f"Train Loss: {tr_loss:.4f} | Dice: {tr_dice:.4f}")
        print(f"Val   Loss: {val_loss:.4f} | Dice: {val_dice:.4f}")

        if val_dice > best_dice:
            best_dice = val_dice
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'history': history,
                'best_dice': best_dice,
            }, best_path)
            print(f"✓ Saved best model (Dice={best_dice:.4f})")

    print("\nBest Val Dice:", best_dice)

    # save history for plotting curves
    hist_df = pd.DataFrame(history)
    hist_df.to_csv(os.path.join(seg_cfg.output_dir,
                                'seg_bm_training_history.csv'),
                   index=False)

if __name__ == "__main__":
    main()


Device: cuda
Original train size: 1177 val size: 326
Train label counts:
 LABEL
0    542
2    355
1    280
Name: count, dtype: int64
Val label counts:
 LABEL
0    137
2    105
1     84
Name: count, dtype: int64
Seg train size (B+M): 822 Seg val size (B+M): 221

Epoch 1/60


Seg Train:   0%|          | 0/103 [00:00<?, ?it/s]

Seg Val:   0%|          | 0/28 [00:00<?, ?it/s]

Train Loss: 1.4457 | Dice: 0.0804
Val   Loss: 1.3593 | Dice: 0.0752
✓ Saved best model (Dice=0.0752)

Epoch 2/60


Seg Train:   0%|          | 0/103 [00:00<?, ?it/s]

...

Seg Val:   0%|          | 0/28 [00:00<?, ?it/s]

Train Loss: 0.4177 | Dice: 0.7248
Val   Loss: 0.4559 | Dice: 0.6823

Epoch 60/60


Seg Train:   0%|          | 0/103 [00:00<?, ?it/s]

Seg Val:   0%|          | 0/28 [00:00<?, ?it/s]

Train Loss: 0.4315 | Dice: 0.7143
Val   Loss: 0.4511 | Dice: 0.6851

Best Val Dice: 0.6857801271336419


In [ ]:
!pip install -q albumentations==1.4.3 segmentation-models-pytorch

import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import f1_score, classification_report, confusion_matrix, balanced_accuracy_score
from tqdm.auto import tqdm
import segmentation_models_pytorch as smp
from sklearn.metrics import balanced_accuracy_score


# ----------------- CONFIG -----------------
class Pipeline2Config:
    img_dir   = 'training_images'
    train_meta = 'metadata_train.csv'
    val_meta   = 'metadata_val.csv'

    output_dir = 'outputs_pipeline2'
    img_size   = 384
    seg_epochs = 60
    cls_epochs = 40
    batch_size = 8
    lr         = 1e-4
    weight_decay = 1e-5
    device     = 'cuda' if torch.cuda.is_available() else 'cpu'
    num_workers = 0

    seed = 42

cfg = Pipeline2Config()
os.makedirs(cfg.output_dir, exist_ok=True)

# ----------------- SEED -----------------
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(cfg.seed)

# ----------------- 3-CLASS SEGMENTATION DATASET -----------------
class NBM3ClassSegDataset(Dataset):
    """
    3-class segmentation: N(background)=0, B=1, M=2
    """
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_name  = row['US']
        mask_name = row['MASK']
        label     = int(row['LABEL'])  # 0=B, 1=M, 2=N

        img_path  = os.path.join(self.img_dir, img_name)
        mask_path = os.path.join(self.img_dir, mask_name)

        # image
        image = Image.open(img_path).convert('L')
        image = np.array(image).astype(np.float32)  # H,W

        # 3-class mask: background=0, lesion=label (B=1 or M=2)
        mask_gt = Image.open(mask_path).convert('L')
        mask_gt = np.array(mask_gt)
        mask_3class = np.zeros_like(mask_gt)
        lesion_pixels = mask_gt > 0
        mask_3class[lesion_pixels] = label  # B=1 or M=2 pixels

        if self.transform:
            aug = self.transform(image=image, mask=mask_3class)
            image = aug['image']
            mask  = aug['mask'].long()
        else:
            image = torch.from_numpy(image).unsqueeze(0)
            mask  = torch.from_numpy(mask_3class).long()

        return image, mask

# ----------------- MASKED B vs M CLASSIFIER DATASET -----------------
class MaskedBMClassifierDataset(Dataset):
    """
    B vs M classification using ground-truth masks during training.
    At test time: will use predicted 3-class seg masks.
    """
    def __init__(self, df, img_dir, transform=None, use_gt_mask=True):
        # only B/M cases
        self.df = df[df['LABEL'].isin([0,1])].reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.use_gt_mask = use_gt_mask

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_name  = row['US']
        mask_name = row['MASK']
        label     = int(row['LABEL'])  # 0=B, 1=M

        img_path  = os.path.join(self.img_dir, img_name)
        mask_path = os.path.join(self.img_dir, mask_name)

        # load image
        image = Image.open(img_path).convert('L')
        image = np.array(image).astype(np.float32)

        # create masked image: multiply by lesion mask
        mask = Image.open(mask_path).convert('L')
        mask = np.array(mask) > 0
        masked_image = image * mask.astype(np.float32)

        # repeat to 3 channels
        masked_image = np.stack([masked_image]*3, axis=2)

        if self.transform:
            aug = self.transform(image=masked_image)
            masked_image = aug['image']

        return masked_image, label

# ----------------- TRANSFORMS -----------------
def get_seg_transforms(phase='train'):
    if phase == 'train':
        return A.Compose([
            A.Resize(cfg.img_size, cfg.img_size),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.2),
            A.RandomRotate90(p=0.3),
            A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.4),
            A.Normalize(mean=[0.0], std=[1.0]),
            ToTensorV2(),
        ])
    else:
        return A.Compose([
            A.Resize(cfg.img_size, cfg.img_size),
            A.Normalize(mean=[0.0], std=[1.0]),
            ToTensorV2(),
        ])

def get_cls_transforms(phase='train'):
    if phase == 'train':
        return A.Compose([
            A.Resize(cfg.img_size, cfg.img_size),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.2),
            A.RandomRotate90(p=0.3),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])
    else:
        return A.Compose([
            A.Resize(cfg.img_size, cfg.img_size),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])

# ----------------- 3-CLASS SEGMENTATION MODEL -----------------
class NBM3ClassUNet(nn.Module):
    def __init__(self, n_channels=1, n_classes=3):
        super().__init__()
        self.model = smp.Unet(
            encoder_name="resnet34",
            encoder_weights="imagenet",
            in_channels=n_channels,
            classes=n_classes,
            activation=None  # logits
        )

    def forward(self, x):
        return self.model(x)

# ----------------- B vs M CLASSIFIER -----------------
class BMClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model(
            'tf_efficientnetv2_s',
            pretrained=True,
            num_classes=0
        )
        in_features = self.backbone.num_features
        self.classifier = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(in_features, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 2)  # B vs M
        )

    def forward(self, x):
        feats = self.backbone(x)
        return self.classifier(feats)

# ----------------- LOSSES & METRICS -----------------
def multiclass_dice_loss(logits, targets, smooth=1e-6):
    """Multiclass Dice loss"""
    probs = torch.softmax(logits, dim=1)
    num_classes = logits.shape[1]

    dice_loss = 0
    for c in range(num_classes):
        pred_c = probs[:, c].unsqueeze(1)
        target_c = (targets == c).float().unsqueeze(1)

        intersection = (pred_c * target_c).sum(dim=(2,3))
        union = pred_c.sum(dim=(2,3)) + target_c.sum(dim=(2,3))
        dice_c = (2 * intersection + smooth) / (union + smooth)
        dice_loss += 1 - dice_c.mean()

    return dice_loss / num_classes

def mean_dice_3class(logits, targets):
    probs = torch.softmax(logits, dim=1)
    num_classes = logits.shape[1]
    dice_scores = []

    for c in range(num_classes):
        pred_c = (probs[:, c] > 0.5).float()
        target_c = (targets == c).float()
        intersection = (pred_c * target_c).sum(dim=(2,3))
        union = pred_c.sum(dim=(2,3)) + target_c.sum(dim=(2,3))
        dice_c = (2 * intersection + 1e-7) / (union + 1e-7)
        dice_scores.append(dice_c.mean())

    return torch.stack(dice_scores).mean().item()

# ----------------- PIPELINE 2 INFERENCE -----------------
class Pipeline2Inference:
    def __init__(self, seg_model_path, cls_model_path, device):
        self.device = device
        self.seg_model = NBM3ClassUNet().to(device)
        self.seg_model.load_state_dict(torch.load(seg_model_path, map_location=device)['model_state_dict'])
        self.seg_model.eval()

        self.cls_model = BMClassifier().to(device)
        self.cls_model.load_state_dict(torch.load(cls_model_path, map_location=device)['model_state_dict'])
        self.cls_model.eval()

    def predict_image_label(self, image_tensor):  # 1x3xHxW
        with torch.no_grad():
            image_tensor = image_tensor.unsqueeze(0).to(self.device)

            # Step 1: 3-class segmentation
            seg_logits = self.seg_model(image_tensor[:, :1])  # take 1st channel
            seg_probs = torch.softmax(seg_logits, dim=1)

            # Check if any B/M lesion pixels
            b_prob = seg_probs[0, 1].sum()  # sum all B pixels
            m_prob = seg_probs[0, 2].sum()  # sum all M pixels

            if b_prob + m_prob < 0.1 * seg_probs[0, 0].sum():  # mostly background
                return 2  # NORMAL

            # Step 2: create masked lesion image
            lesion_mask = (seg_probs[0, 1] + seg_probs[0, 2] > 0.3).float()
            masked_img = image_tensor[0] * lesion_mask.unsqueeze(0)

            # Step 3: B vs M classification
            cls_logits = self.cls_model(masked_img.unsqueeze(0))
            pred_cls = cls_logits.argmax(1).item()

            return pred_cls.item()  # 0=B, 1=M

# ----------------- MAIN TRAINING -----------------
def train_3class_seg():
    print("=== Training 3-class N/B/M Segmentation ===")

    train_df = pd.read_csv(cfg.train_meta)
    val_df   = pd.read_csv(cfg.val_meta)

    train_ds = NBM3ClassSegDataset(train_df, cfg.img_dir, get_seg_transforms('train'))
    val_ds   = NBM3ClassSegDataset(val_df, cfg.img_dir, get_seg_transforms('val'))

    train_loader = DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=False, num_workers=0)

    model = NBM3ClassUNet().to(cfg.device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=cfg.seg_epochs)

    best_dice = 0
    history = {'train_loss': [], 'val_loss': [], 'val_dice': []}

    for epoch in range(cfg.seg_epochs):
        model.train()
        train_loss = 0
        for imgs, masks in tqdm(train_loader, desc='Train'):
            imgs, masks = imgs.to(cfg.device), masks.to(cfg.device)
            optimizer.zero_grad()
            logits = model(imgs)
            loss = criterion(logits, masks)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # validation
        model.eval()
        val_loss, val_dice_total = 0, 0
        with torch.no_grad():
            for imgs, masks in tqdm(val_loader, desc='Val'):
                imgs, masks = imgs.to(cfg.device), masks.to(cfg.device)
                logits = model(imgs)
                loss = criterion(logits, masks)
                val_loss += loss.item()
                val_dice_total += mean_dice_3class(logits, masks)

        val_dice = val_dice_total / len(val_loader)
        scheduler.step()

        print(f"Epoch {epoch+1}: Train Loss: {train_loss/len(train_loader):.4f}, "
              f"Val Loss: {val_loss/len(val_loader):.4f}, Val Dice: {val_dice:.4f}")

        if val_dice > best_dice:
            best_dice = val_dice
            torch.save({'model_state_dict': model.state_dict(), 'best_dice': best_dice},
                      os.path.join(cfg.output_dir, 'best_3class_seg.pth'))

    return os.path.join(cfg.output_dir, 'best_3class_seg.pth')

def train_bm_classifier(seg_model_path):
    print("=== Training B vs M masked classifier ===")

    train_df = pd.read_csv(cfg.train_meta)
    val_df   = pd.read_csv(cfg.val_meta)

    train_ds = MaskedBMClassifierDataset(train_df, cfg.img_dir, get_cls_transforms('train'))
    val_ds   = MaskedBMClassifierDataset(val_df, cfg.img_dir, get_cls_transforms('val'))

    train_loader = DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=False, num_workers=0)

    model = BMClassifier().to(cfg.device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=cfg.cls_epochs)

    best_f1 = 0
    for epoch in range(cfg.cls_epochs):
        model.train()
        train_loss = 0
        train_preds, train_labels = [], []
        for imgs, labels in tqdm(train_loader, desc='Train'):
            imgs, labels = imgs.to(cfg.device), labels.to(cfg.device)
            optimizer.zero_grad()
            logits = model(imgs)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            preds = logits.argmax(1)
            train_preds.extend(preds.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

        # validation
        model.eval()
        val_loss, val_preds, val_labels = 0, [], []
        with torch.no_grad():
            for imgs, labels in tqdm(val_loader, desc='Val'):
                imgs, labels = imgs.to(cfg.device), labels.to(cfg.device)
                logits = model(imgs)
                loss = criterion(logits, labels)
                val_loss += loss.item()
                preds = logits.argmax(1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_f1 = f1_score(val_labels, val_preds, average='macro')
        scheduler.step()

        print(f"Epoch {epoch+1}: Train Loss: {train_loss/len(train_loader):.4f}, "
              f"Val Loss: {val_loss/len(val_loader):.4f}, Val F1: {val_f1:.4f}")

        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save({'model_state_dict': model.state_dict(), 'best_f1': best_f1},
                      os.path.join(cfg.output_dir, 'best_bm_classifier.pth'))

    return os.path.join(cfg.output_dir, 'best_bm_classifier.pth')

# ----------------- EVALUATION & COMPARISON -----------------
def evaluate_pipeline2(seg_model_path, cls_model_path, val_df, img_dir):
    """Run full pipeline 2 on validation set and return image-level predictions"""
    print("=== Evaluating Pipeline 2 on validation set ===")

    infer = Pipeline2Inference(seg_model_path, cls_model_path, cfg.device)

    val_ds = BreastUSDataset(val_df, img_dir, transform=get_transforms('val'))
    val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=0)

    predictions = []
    true_labels = []

    infer.seg_model.eval()
    infer.cls_model.eval()

    with torch.no_grad():
        for image, true_label in tqdm(val_loader, desc='Pipeline 2 inference'):
            pred_label = infer.predict_image_label(image)
            predictions.append(pred_label)
            true_labels.append(true_label.item())

    # Compute metrics
    balanced_acc = balanced_accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average='macro')
    cm = confusion_matrix(true_labels, predictions, labels=[0,1,2])

    print(f"Pipeline 2 - Balanced Accuracy: {balanced_acc:.4f}")
    print(f"Pipeline 2 - Macro F1: {f1:.4f}")
    print("Pipeline 2 - Confusion Matrix:")
    print(cm)

    return predictions, true_labels, balanced_acc, f1, cm

# ----------------- MAIN EXECUTION -----------------
if __name__ == "__main__":
    print("Pipeline 2: 3-class Segmentation → B vs M Classifier")

    # Step 1: Train 3-class segmentation
    seg_model_path = train_3class_seg()

    # Step 2: Train B vs M classifier on masked lesions
    cls_model_path = train_bm_classifier(seg_model_path)

    # Step 3: Evaluate full pipeline on validation set
    val_df = pd.read_csv(cfg.val_meta)
    predictions, true_labels, bal_acc, f1, cm = evaluate_pipeline2(
        seg_model_path, cls_model_path, val_df, cfg.img_dir
    )

    # Save pipeline results for report comparison
    results = {
        'seg_model_path': seg_model_path,
        'cls_model_path': cls_model_path,
        'balanced_accuracy': bal_acc,
        'macro_f1': f1,
        'confusion_matrix': cm.tolist(),
        'predictions': predictions,
        'true_labels': true_labels
    }
    torch.save(results, os.path.join(cfg.output_dir, 'pipeline2_results.pth'))

    print("\n✓ Pipeline 2 complete!")
    print(f"Models saved in: {cfg.output_dir}")
    print("Ready for report comparison with Pipeline 1!")



Pipeline 2: 3-class Segmentation → B vs M Classifier
=== Training 3-class N/B/M Segmentation ===


Train:   0%|          | 0/148 [00:00<?, ?it/s]

Val:   0%|          | 0/41 [00:00<?, ?it/s]

IndexError: Dimension out of range (expected to be in range of [-3, 2], but got 3)